In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3, MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array

# Constants
dataset_path = 'trainingdataset'
klass_dir = 'test'
img_size = 299  # Input size for both InceptionV3 and MobileNetV2
batch_size = 32

# Load dataset
def load_dataset():
    images = []
    labels = []
    classes = os.listdir(dataset_path)
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):  # Check if the current item is a directory
            for image_name in os.listdir(class_path):
                image_path = os.path.join(class_path, image_name)
                try:
                    image = cv2.imread(image_path)
                    if image is not None:  # Check if the image was loaded successfully
                        image = cv2.resize(image, (img_size, img_size))
                        images.append(image)
                        labels.append(class_name)
                    else:
                        print(f"Skipping invalid image: {image_path}")
                except Exception as e:
                    print(f"Error loading image: {image_path}")
                    print(str(e))
    return images, labels

# Preprocess dataset
def preprocess_dataset(images, labels):
    images = np.array(images)
    images = images.astype('float32') / 255.0

    # Convert labels to integers
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)

    return images, labels, label_encoder

# Load dataset
images, labels = load_dataset()

# Preprocess dataset
images, labels, label_encoder = preprocess_dataset(images, labels)

# Split dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load InceptionV3 model pre-trained on ImageNet dataset
inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
mobilenet_base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Add custom classification head
inception_output = inception_base_model.output
inception_output = GlobalAveragePooling2D()(inception_output)
mobilenet_output = mobilenet_base_model.output
mobilenet_output = GlobalAveragePooling2D()(mobilenet_output)

# Concatenate the outputs of InceptionV3 and MobileNetV2
concatenated_output = concatenate([inception_output, mobilenet_output])

# Add custom dense layers for classification
x = Dense(128, activation='relu')(concatenated_output)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Combine base models and custom head
model = Model(inputs=[inception_base_model.input, mobilenet_base_model.input], outputs=predictions)

# Freeze base model layers (optional)
for layer in inception_base_model.layers:
    layer.trainable = False
for layer in mobilenet_base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit([train_images, train_images], train_labels, batch_size=batch_size, epochs=30, validation_data=([test_images, test_images], test_labels))

# Plot accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

# Evaluate the model
loss, accuracy = model.evaluate([test_images, test_images], test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
predictions = model.predict([test_images, test_images])
predicted_labels = np.argmax(predictions, axis=1)

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, predicted_labels)

# Plot confusion matrix as an image
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Calculate F1 score
f1 = f1_score(test_labels, predicted_labels, average='macro')

# Display F1 score
print(f'F1 Score: {f1:.4f}')

# Generate classification report
class_report = classification_report(test_labels, predicted_labels, target_names=label_encoder.classes_, output_dict=True)

# Calculate sensitivity and specificity
sensitivity = class_report['macro avg']['recall']
specificity = (conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1]))  # Specificity for the first class

# Display classification report
print(class_report)

# Display sensitivity and specificity
print(f'Sensitivity: {sensitivity:.4f}')
print(f'Specificity: {specificity:.4f}')

# Save the model
model.save('hybrid_inception_mobilenet.h5')

# Directory containing images to be classified
klass_dir = 'test'

# Load and preprocess the images
images_to_be_classified = []
original_images = []
for image_name in os.listdir(klass_dir):
    # Construct the full path to the image
    image_path = os.path.join(klass_dir, image_name)
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    if image is not None:  # Check if the image was loaded successfully
        original_images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Save original image for display
        # Resize the image
        resized_image = cv2.resize(image, (img_size, img_size))
        # Preprocess the image
        preprocessed_image = resized_image / 255.0
        images_to_be_classified.append(preprocessed_image)
    else:
        print(f"Skipping invalid image: {image_path}")

images_to_be_classified = np.array(images_to_be_classified)

# Make predictions
predictions = model.predict([images_to_be_classified, images_to_be_classified])

# Decode predictions
predicted_labels = np.argmax(predictions, axis=1)
predicted_classes = label_encoder.inverse_transform(predicted_labels)

# Print predictions
for i in range(len(images_to_be_classified)):
    image_name = os.listdir(klass_dir)[i]
    prediction = predicted_classes[i]
    # Plot the image with predicted label
    plt.figure(figsize=(5, 5))
    plt.imshow(original_images[i])
    plt.title(f'True Label: {image_name}, Predicted: {prediction}')
    plt.axis('off')
    plt.show()